<a href="https://colab.research.google.com/github/naseehasalam/IRIS_gyanrecs/blob/main/gyan_recs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python Program to perform summarization on the CNN/Dailymail Dataset**
_I have used the csv version of the test data from a Kaggle competition instead of the given CNN/Dailymail dataset URL'S which were provided in the Recruitments document since I couldnt not scrape the article from them properly_

In the following python program we perform the tasks given below:-





# 1. Data from the CSV file is read using Pandas library

In [ ]:
import pandas as pd

df=pd.read_csv("test.csv")

article_text=list(df['article'])


#2. The Read data is tokenised using the NLTK library to words and sentences and further filtered.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize

stop_words=list(stopwords.words('english'))

sentences={}
for i in range(len(article_text)):
  sent=sent_tokenize(article_text[i])
  sentences[i]=sent

words={}
for i in range(len(article_text)):
  word=word_tokenize(article_text[i])
  words[i]=word

for i in words:
  for j in words[i]:
    if j in stop_words:
      words[i].remove(j)



#3. Transformers Library is Installed.

In [ ]:
!pip install transformers

#4.Importing pipelines from the Hugging Face Transformer.

In [2]:
import transformers
from transformers import pipeline



#5.Chunking the data so that the pipeline is stable

In [ ]:
max_chunk=500
current_chunk=0
chunks=[]

for i in sentences:
  for j in sentences[i]:
    if len(chunks)==current_chunk+1:
      if len(chunks[current_chunk]) + len(j.split(' ')) <= max_chunk:
        chunks[current_chunk].extend(j.split(' '))
      else:
        current_chunk+=1
        chunks.append(j.split(' '))
    else:
      print(current_chunk)
      chunks.append(j.split(' '))

#6. The summarization pipeline is created.

In [ ]:
summariser=pipeline("summarization")

#7.Summarising each article.

In [ ]:
summaries=[]
for i in chunks:
  summ=summariser(i,max_length=150,min_length=30,do_sample=False)
  summaries.append(summ)

sum2=[]
sum_pd=pd.DataFrame(summaries,column=['summary'])
sum_pd.to_csv('test.csv',index=False)
  

#8.ROGUE score calculation


In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

from collections import Counter

df=pd.read_csv("test.csv")

article_text=list(df['article'])
summaries=list(df['summary'])

for i in article_text:
  for j in summaries:
    scores = rouge.get_scores(j,i)
scores